# Use Keras to recognize hand-written digits with `ibm-watson-machine-learning`

This notebook contains steps and code to demonstrate support of Deep Learning experiments in the Watson Machine Learning service. It introduces commands for data retrieval, training definition persistance, experiment training, model persistance, and model deployment. 

Some familiarity with Python is helpful. This notebook uses Python 3.7.


## Learning goals

The learning goals of this notebook are:

-  Working with the Watson Machine Learning service.
-  Training Deep Learning models (Keras - TensorFlow).
-  Saving trained models in Watson Machine Learning repository.
-  Online deployment of the trained model.
-  Scoring two samples

## Contents

This notebook contains the following parts:

1.	[Setup](#setup)
2.	[Create model definition](#model_df)
3.	[Train model](#training)
4.  [Persist trained model](#persist)
5.	[Deploy and Score](#deploy)
6.  [Clean up](#clean)
7.	[Summary and next steps](#summary)

<a id="setup"></a>
## 1. Setup



### 1.1 Working with Cloud Object Storage

The `ibm_boto3` library allows Python developers to manage Cloud Object Storage.

**Note:** If `ibm_boto3` is not preinstalled in you environment please install it by running the following command: `!pip install ibm-cos-sdk`

In [ ]:
import ibm_boto3
from ibm_botocore.client import Config
import os
import json
import warnings
warnings.filterwarnings('ignore')

**Action**: Paste your COS credentials in the following cell.  

In [ ]:
cos_credentials = PASTE CREDENTIALS HERE    


**Action**: Paste your service endpoint 

In [ ]:
service_endpoint = "https://PASTE SERVICE ENDPOINT HERE"



Create the Boto resource by providing type, endpoint_url and credentials.

In [ ]:
cos = ibm_boto3.resource('s3',
                         ibm_api_key_id=cos_credentials["apikey"],
                         ibm_service_instance_id=cos_credentials["resource_instance_id"],
                         ibm_auth_endpoint="https://iam.ng.bluemix.net/oidc/token",
                         config=Config(signature_version='oauth'),
                         endpoint_url=service_endpoint)


Create the buckets that you will use to store training data and training results.

**Note:**: Bucket name has to be unique - please update following ones to any unique name.<br>
Replace xxx with your initials below in lower case

In [ ]:
buckets = ['tf-keras-data-example-xxx', 'tf-keras-results-example-xxx']
for bucket in buckets:
    if not cos.Bucket(bucket) in cos.buckets.all():
        print('Creating bucket "{}"...'.format(bucket))
        try:
            cos.create_bucket(Bucket=bucket)
        except ibm_boto3.exceptions.ibm_botocore.client.ClientError as e:
            print('Error: {}.'.format(e.response['Error']['Message']))

The buckets are created.

In [ ]:
print(list(cos.buckets.limit(50)))

### 1.2 Download the MNIST data and upload it to the COS bucket

In this notebook we work with the Keras **MNIST** sample dataset. Download the training data. 

Following cell creates the 'MNIST_KERAS_DATA' folder and downloads the file from link.

**Note:** First install `wget` library by the following command
`!pip install wget`

In [ ]:
link = 'https://s3.amazonaws.com/img-datasets/mnist.npz'

In [ ]:
!pip install wget
import wget

data_dir = 'MNIST_KERAS_DATA'
if not os.path.isdir(data_dir):
    os.mkdir(data_dir)

if not os.path.isfile(os.path.join(data_dir, os.path.join(link.split('/')[-1]))):
    wget.download(link, out=data_dir)  
        
!ls MNIST_KERAS_DATA

We will now upload the training data to the bucket assigned for training

In [ ]:
bucket_name = buckets[0]
bucket_obj = cos.Bucket(bucket_name)

for filename in os.listdir(data_dir):
    with open(os.path.join(data_dir, filename), 'rb') as data: 
        bucket_obj.upload_file(os.path.join(data_dir, filename), filename)
        print('{} is uploaded.'.format(filename))

Here are the list of objects in the training bucket

In [ ]:
for obj in bucket_obj.objects.all():
    print('Object key: {}'.format(obj.key))
    print('Object size (kb): {}'.format(obj.size/1024))
    

### 1.3 Connection to WML

Authenticate the Watson Machine Learning service on IBM Cloud. You need to provide the platform `api_key` and instance `location`.

Refer to the Lab-5 notebook (Heart Disease) to get the api key and location. 

In [ ]:
api_key = 'PASTE API KEY HERE'
location = 'PASTE LOCATION HERE'

In [ ]:
wml_credentials = {
    "apikey": api_key,
    "url": 'https://' + location + '.ml.cloud.ibm.com'
}

### Install and import the `ibm-watson-machine-learning` package
**Note:** `ibm-watson-machine-learning` documentation can be found <a href="http://ibm-wml-api-pyclient.mybluemix.net/" target="_blank" rel="noopener no referrer">here</a>.

In [ ]:
!pip install -U ibm-watson-machine-learning

In [ ]:
from ibm_watson_machine_learning import APIClient

client = APIClient(wml_credentials)

### Working with spaces

We created a deployment space in Lab-1. The code below provides a list
of deployment spaces. The space_id is obtained from the ID column. 


In [ ]:
client.spaces.list(limit=10)

Copy the value in the ID column that corresponds to the Watson Studio Labs deployment space

In [ ]:
space_id = "PASTE SPACE ID HERE"

To be able to interact with all resources available in Watson Machine Learning, you need to set the **space** which you will be using.  

In [ ]:
client.set.default_space(space_id)

The model is ready to be trained.

<a id="model_def"></a>
# 2. Create model definition

For the purpose of this example two Keras model definitions have been prepared:

 - Multilayer Perceptron (MLP)
 - Convolution Neural Network (CNN)
 
We will train the Convolution Neural Network below. 

### 2.1 Prepare model definition metadata

In [ ]:
metaprops = {
    client.model_definitions.ConfigurationMetaNames.NAME: "MNIST cnn model definition",
    client.model_definitions.ConfigurationMetaNames.DESCRIPTION: "MNIST cnn model definition",
    client.model_definitions.ConfigurationMetaNames.COMMAND: "python3 mnist_cnn.py --trainImagesFile ${DATA_DIR}/train-images-idx3-ubyte.gz --trainLabelsFile ${DATA_DIR}/train-labels-idx1-ubyte.gz --testImagesFile ${DATA_DIR}/t10k-images-idx3-ubyte.gz --testLabelsFile ${DATA_DIR}/t10k-labels-idx1-ubyte.gz --learningRate 0.001 --trainingIters 6000",
    client.model_definitions.ConfigurationMetaNames.PLATFORM: {"name": "python", "versions": ["3.6"]},
    client.model_definitions.ConfigurationMetaNames.VERSION: "2.0",
    client.model_definitions.ConfigurationMetaNames.SPACE_UID: space_id
}

### 2.2 Get sample model definition content files from git (Python scripts with CNN and MLP)

The MNIST.zip file contains 3 python files (1) multilayer perceptron model, (2) convolution neural network model, (3) metrics file

In [ ]:
filename_mnist = 'MNIST.zip'

if not os.path.isfile(filename_mnist):
    filename_mnist = wget.download('https://github.com/IBM/watson-machine-learning-samples/raw/master/definitions/keras/mnist/MNIST.zip')

### 2.3 Publish model definition

In [ ]:
model_definition_details = client.model_definitions.store(filename_mnist, meta_props=metaprops)

In [ ]:
model_definition_id = client.model_definitions.get_id(model_definition_details)
print(model_definition_id)

#### List model definitions

In [ ]:
client.model_definitions.list(limit=5)

<a id="training"></a>
# 3. Train model

### 3.1 Prepare training metadata

The code below provides a training manifest that is used by the Watson Machine Learning service to create the environment to run the neural network training. The training data bucket, the results bucket, the command to run the Keras model, and the environment is specified. This is referred to as a training definition. 

In [ ]:
training_metadata = {
    client.training.ConfigurationMetaNames.NAME: "Keras-MNIST",
    client.training.ConfigurationMetaNames.SPACE_UID: space_id,
    client.training.ConfigurationMetaNames.DESCRIPTION: "Keras-MNIST predict written digits",
    client.training.ConfigurationMetaNames.TAGS :[{
      "value": "MNIST",
      "description": "predict written digits"
    }],
    client.training.ConfigurationMetaNames.TRAINING_RESULTS_REFERENCE:  {
    "name": "MNIST results",
    "connection": {
            "endpoint_url": service_endpoint,
            "access_key_id": cos_credentials['cos_hmac_keys']['access_key_id'],
            "secret_access_key": cos_credentials['cos_hmac_keys']['secret_access_key']
      },
      "location": {
        "bucket": buckets[0]
      },
    "type": "s3"
  },
  client.training.ConfigurationMetaNames.MODEL_DEFINITION:{
        "id": model_definition_id,
        "command": "python3 mnist_cnn.py --trainImagesFile ${DATA_DIR}/train-images-idx3-ubyte.gz --trainLabelsFile ${DATA_DIR}/train-labels-idx1-ubyte.gz --testImagesFile ${DATA_DIR}/t10k-images-idx3-ubyte.gz --testLabelsFile ${DATA_DIR}/t10k-labels-idx1-ubyte.gz --learningRate 0.001 --trainingIters 6000",
        "hardware_spec": {
          "name": "K80",
          "nodes": 1
        },
        "software_spec": {
          "name": "tensorflow_1.15-py3.6"
        },
        "parameters": {
          "name": "MNIST cnn",
          "description": "Simple MNIST cnn model"
        }
      },
  client.training.ConfigurationMetaNames.TRAINING_DATA_REFERENCES: [
       {
      "name": "training_input_data",
      "type": "s3",
      "connection": {
        "endpoint_url": service_endpoint,
        "access_key_id": cos_credentials['cos_hmac_keys']['access_key_id'],
        "secret_access_key": cos_credentials['cos_hmac_keys']['secret_access_key']
      },
      "location": {
        "bucket": buckets[1]
      },
      "schema": {
        "id":"idcnn_schema",
        "fields": [
          {
            "name": "text",
            "type": "string"
          }
        ]
      }
    }
  ]
}

### 3.2 Train model in background

Submit the training run to the Watson Machine Learning Service. This is an asynchronous call. 

In [ ]:
training = client.training.run(training_metadata)

### 3.3 Get training id and status

In [ ]:
training_id = client.training.get_uid(training)

You will needed to repeatedly execute the following code to monitor the status of the training process. The status should go from pending, to running, to completed. Don't proceed with the notebook until you have a status of completed. It should take about 5 minutes. 

In [ ]:
client.training.get_status(training_id)["state"]

### 3.4 Get training details

After executing the code in the next cell, scroll down until you see ml_metrics;  you should see ml_metrics listed 4 times. 
The accuracy of the training run corresponds to the 4th ml_metrics listed. It should be 98.1% accuracy on training and 98.85% accuracy on the validation data. 

In [ ]:
training_details = client.training.get_details(training_id)
print(json.dumps(training_details, indent=2))

#### List trainings

In [ ]:
client.training.list(limit=5)

#### Cancel training

You can cancel the training run by calling the method below. Otherwise skip past it. 
**Tip**: If you want to delete train runs and results add `hard_delete=True` as a parameter.

<a id="persist"></a>
# 4. Persist trained model

### 4.1 Download trained model from COS

In [ ]:
uid = client.training.get_details(training_id)['entity']['results_reference']['location']['logs']

#### Download model from COS

In [ ]:
bucket_name = buckets[0]
bucket_obj = cos.Bucket(bucket_name)

model_path = ""
for obj in bucket_obj.objects.iterator():
    if uid in obj.key and obj.key.endswith(".h5"):
        model_path = obj.key
        break

model_name = model_path.split("/")[-1]
bucket_obj.download_file(model_path, model_name)

#### Unpack model and compress it to tar.gz format

In [ ]:
import tarfile
    
model_name = "mnist_cnn.h5"    
with tarfile.open(model_name + ".tar.gz", "w:gz") as tar:
    tar.add("mnist_cnn.h5")

### 4.2 Publish model

In [ ]:
software_spec_uid = client.software_specifications.get_uid_by_name('tensorflow_1.15-py3.6')

In [ ]:
model_meta_props = {
                    client.repository.ModelMetaNames.NAME: "Keras MNIST",
                    client.repository.ModelMetaNames.TYPE: "keras_2.2.5",
                    client.repository.ModelMetaNames.SOFTWARE_SPEC_UID: software_spec_uid
}

published_model = client.repository.store_model(model='mnist_cnn.h5.tar.gz', meta_props=model_meta_props)
model_uid = client.repository.get_model_uid(published_model)

### 4.3 Get model details

In [ ]:
model_details = client.repository.get_details(model_uid)
print(json.dumps(model_details, indent=2))

#### List stored models

In [ ]:
client.repository.list_models(limit=5)

<a id="deploy"></a>
# 5. Deploy and score

### 5.1 Create online deployment for published model

In [ ]:
deployment = client.deployments.create(model_uid, meta_props={
                                            client.deployments.ConfigurationMetaNames.NAME: "Keras MNIST",
                                            client.deployments.ConfigurationMetaNames.ONLINE: {}})

deployment_uid = client.deployments.get_uid(deployment)

### 5.2 Get deployments details

In [ ]:
deployments_details = client.deployments.get_details(deployment_uid)
print(json.dumps(deployments_details, indent=2))

#### List deployments

In [ ]:
client.deployments.list(limit=5)

### 5.3 Score deployed model

Let's plot two digits. **Action:** Please install `matplotlib`, `numpy`

In [ ]:
import wget

dataset_filename='mnist.npz'

if not os.path.isfile(dataset_filename):
    dataset_filename = wget.download('https://github.com/IBM/watson-machine-learning-samples/raw/master/data/mnist/mnist.npz')

In [ ]:
import numpy as np

mnist_dataset = np.load(dataset_filename)
x_test = mnist_dataset['x_test']

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
for i, image in enumerate([x_test[0], x_test[1]]):
    plt.subplot(2, 2, i + 1)
    plt.axis('off')
    plt.imshow(image, cmap=plt.cm.gray_r, interpolation='nearest')

Our input node expects to get data with shape (28,28,1) so we need to reshape our two digits.

In [ ]:
image_1 = x_test[0]/ 255
image_2 = x_test[1]/ 255
image_1 = image_1.reshape(28,28,1)
image_2 = image_2.reshape(28,28,1)
    


#### Prepare scoring payload and score.

In [ ]:
scoring_payload = {
    client.deployments.ScoringMetaNames.INPUT_DATA : [
        {'values': [image_1.tolist(), image_2.tolist()]}
    ]
}
scores = client.deployments.score(deployment_uid, meta_props=scoring_payload)
print("Scoring result:\n" + json.dumps(scores, indent=2))

<a id="clean"></a>
# 6. Clean up

If you want to clean up all created assets:
- experiments
- trainings
- pipelines
- model definitions
- models
- functions
- deployments

please follow up this sample [notebook](https://github.com/IBM/watson-machine-learning-samples/blob/master/notebooks/python_sdk/instance-management/Machine%20Learning%20artifacts%20management.ipynb).

<a id="summary"></a>
# 7. Summary and next steps     

 You successfully completed this notebook! You learned how to use `ibm-watson-machine-learning-client` to run experiments. Check out our _[Online Documentation](https://console.ng.bluemix.net/docs/services/PredictiveModeling/index.html)_ for more samples, tutorials, documentation, how-tos, and blog posts. 

### Author

**Jan Sołtysik**, Intern in Watson Machine Learning.

Modified by Bernard Beekman, IBM Federal and Public Sector

Copyright © 2020 IBM. This notebook and its source code are released under the terms of the MIT License.